In [40]:
import pandas as pd
import yfinance as yf
from collections.abc import Collection
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import numpy as np
from datetime import datetime, date, timedelta

In [ ]:
coints_df = pd.read_csv('../data/potential_pairs.csv')
coints_df = coints_df[coints_df['pvalue'] < 0.02]
coints_df['stock1'] = coints_df['pair'].str.split('_', expand=True)[0]
coints_df['stock2'] = coints_df['pair'].str.split('_', expand=True)[1]
pairs = list(zip(coints_df['stock1'], coints_df['stock2']))

In [ ]:

# TODO 
# 1) this probably should pull stored values esp for back testing
# 2) this probably should record the earliest date both assets have a price
def get_adj_close_history_df(tickers: list[str], start_date: str, end_date: str) -> pd.DataFrame:
    price_df = yf.download(
        tickers,
        start=start_date,
        end=end_date,
        progress=False,
        auto_adjust=True
    )['Close']
    return price_df.dropna()

# NOTE: This methodology came from Caldeira & Caldeira 2013. Paper is in references folder
def is_price_series_integrated_of_order_one(
        price_series: pd.Series,
        pvalue_threshold: float = 0.05
) -> bool:
    p_level = adfuller(price_series)[1]
    # Test if series is stationary
    if p_level < pvalue_threshold:
        return False

    # Test if series is stationary after applying one difference
    p_diff = adfuller(price_series.diff().dropna())[1]
    return True if p_diff < pvalue_threshold else False


def is_pair_engle_granger_cointegrated(
        ticker1_price_series: pd.Series,
        ticker2_price_series: pd.Series,
        pvalue_threshold: float = 0.05
) -> bool:
    try:
        _, pvalue, _ = coint(ticker1_price_series, ticker2_price_series)
    except (ZeroDivisionError, ValueError):
        print(
            f"Asset pair: {ticker1_price_series.name}/{ticker2_price_series.name} unable to be cointegrated"
            )
    return True if pvalue < pvalue_threshold else False

# TODO underwrite det_order and k_ar_diff parameter values
# this are just from chatgpt
def is_pair_johansen_cointegrated(
        pair_price_history_df: pd.DataFrame
) -> bool:
    johansen_res = coint_johansen(pair_price_history_df.values, det_order=0, k_ar_diff=1)

    # Trace statistic for H0: rank = 0
    test_stat = johansen_res.lr1[0]
    
    
def is_pair_tradable(
        ticker_pair: tuple[str],
        start_date: str,
        end_date: str
) -> bool:
    pair_price_history_df = get_adj_close_history_df(ticker_pair, start_date, end_date)
    ticker1_price_series = pair_price_history_df.iloc[:,0]
    ticker2_price_series = pair_price_history_df.iloc[:,1]
    if not is_price_series_integrated_of_order_one(ticker1_price_series):
        return False
    if not is_price_series_integrated_of_order_one(ticker2_price_series):
        return False
    if not is_pair_engle_granger_cointegrated(ticker1_price_series, ticker2_price_series):
        return False
    
    
    

    

# TODO should eventually create a generic abstraction that takes
# a universe (list of tickers) and returns a collection of tradable pairs.
# But that computation is time consuming and I already have a list of candidate pairs
# so I'm going to start there for now.

for pair in pairs:
    start_date = '2015-1-1'
    end_date = '2020-1-1'
    if is_pair_tradable(pair, start_date, end_date):
        # create signal



            
            





In [26]:
df = yf.download(
        ["AAPL","MSFT"],
        start='2015-1-1',
        end='2020-1-1',
        progress=False
    )['Close']

/var/folders/9t/v6gpfcbs71bcpv1bsh004rrc0000gn/T/ipykernel_88009/1000552063.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(


In [39]:
df


Ticker,AAPL,MSFT
Date,,
2015-01-02,24.261045,39.933048
2015-01-05,23.577574,39.565849
2015-01-06,23.579796,38.985119
2015-01-07,23.910429,39.480438
2015-01-08,24.829128,40.641865
...,...,...
2019-12-24,68.654984,149.709076
2019-12-26,70.017097,150.936172
2019-12-27,69.990524,151.212021


In [35]:
df.iloc[:,0].name



'AAPL'